In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
def load(url): 
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )
    docs = loader.load()

    return docs[0]

In [3]:
def create_retriever():
    urls = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]
    
    docs = []
    
    for url in urls:
        docs.append(load(url))
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    return retriever

In [4]:
def relevance_checker(question, retriever):
    retriver_result = retriever.invoke(question)
    query = "question: " + question + "\n\n"
    query += "content: " + str(retriver_result)

    parser = JsonOutputParser()
    prompt = PromptTemplate(
        template="""
유저의 question과 content가 들어오면, question과 content가 서로 관련이 있는지 알려줘.
관련이 있다면, yes
관련이 없다면, no
답변해줘

Answer the user query.\n{format_instructions}\n{query}\n
key는 related로 해줘
    """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    result = chain.invoke({"query": query})
    return result, str(retriver_result)

In [5]:
def generate_answer(question, docs):
    query = "question: " + question + "\n\n"
    query += "content: " + docs
    
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "content를 참고하여 question의 답변을 작성해줘"
            }, 
            {
                "role": "user",
                "content": query
            }
        ]
    )

    return response.choices[0].message.content

In [6]:
def hallucination_answer(answer, docs):
    parser = JsonOutputParser()
    
    query = "answer: " + answer + "\n\n"
    query += "docs: " + docs
    
    prompt = PromptTemplate(
        template="""
유저의 answer과 docs가 들어오면, docs에 근거하여 답변이 잘 작성되었는지 확인해줘.
docs에 없는 내용을 지어냈는지 확인해야해.
말을 지어냈다면,  yes
docs에 근거했다면, no
답변해줘

Answer the user query.\n{format_instructions}\n{query}\n
key 값은 hallucination으로 해줘
    """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    result = chain.invoke({"query": query})

    return result

In [7]:
def final(answer, docs):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "content에서 source를 뽑아서, [answer 내용]\n source: [source]형태로 적어줘"
            }, 
            {
                "role": "user",
                "content": 'answer: ' + answer + '\ndocs:' + docs
            }
        ]
    )

    print(response.choices[0].message.content)

In [8]:
def main(question):
    retriever = create_retriever()
    print("## relevance check ##")
    result, docs = relevance_checker(question, retriever)
    print(result)
    print()

    if result['related'] == 'yes':
        for i in range(3):
            print("## generate answer ##")
            answer = generate_answer(question, docs)
            print(answer)
            print()
    
            print("## check hallucination ##")
            result = hallucination_answer(answer, docs)
            print(result)
            print()

            if result['hallucination'] == 'no':
                print("## Fianl ##")
                answer_fin = final(answer, docs)
                return answer_fin
    else:
        print("검색 결과가 없습니다.")
        return False

In [9]:
main("agent memory")

## relevance check ##
{'related': 'yes'}

## generate answer ##
Agent memory can be categorized into short-term and long-term memory components. 

1. **Short-Term Memory**: This involves in-context learning, which allows the model to retain information temporarily while processing tasks. It is utilized during immediate interactions within a specific context.

2. **Long-Term Memory**: This type of memory provides the capability to retain and recall extensive information over extended periods, often through an external database or vector store that allows for efficient retrieval of information when needed. 

In addition to these types, there are mechanisms such as the memory stream, which tracks observations and events from the agent's experiences in natural language. This stream can inform the agent's behavior based on relevance, recency, and importance when recalling past experiences. Another key aspect is the reflection mechanism, which enables agents to synthesize memories into highe

In [10]:
main("I like apple")

## relevance check ##
{'related': 'no'}

검색 결과가 없습니다.


False

In [11]:
main("What is CoT")

## relevance check ##
{'related': 'yes'}

## generate answer ##
CoT, or Chain-of-Thought, is a prompting technique used in artificial intelligence, particularly in large language models. Introduced by Wei et al. in 2022, CoT prompting involves generating a series of short sentences that outline reasoning logics step by step, known as reasoning chains or rationales. This method is particularly beneficial for solving complex reasoning tasks, especially when using large models with over 50 billion parameters. In contrast, simpler tasks see only marginal improvements from CoT prompting. There are two main types of CoT prompts, one of which includes few-shot CoT, where models are prompted with a few examples of high-quality reasoning chains, whether manually crafted or generated by the model itself.

## check hallucination ##
{'hallucination': 'no'}

## Fianl ##
CoT, or Chain-of-Thought, is a prompting technique used in artificial intelligence, particularly in large language models. Introdu